Note: The script uses Berkeley Neural Parser to parse the generated instructions, and visualize the results using Plotly.

Please make sure to install benepar following their documentation [here](https://github.com/nikitakit/self-attentive-parser#installation).

In [1]:
import random
import benepar, spacy
nlp = spacy.load('en_core_web_md')
doc = nlp("The time for action is now. It's never too late to do something.")

if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})

/home/luxinyu2021/miniconda3/envs/visualize/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
def find_root_verb_and_its_dobj(tree_root):
    # first check if the current node and its children satisfy the condition
    if tree_root.pos_ == "VERB":
        for child in tree_root.children:
            if child.dep_ == "dobj" and child.pos_ == "NOUN":
                return tree_root.lemma_, child.lemma_
        return tree_root.lemma_, None
    # if not, check its children
    for child in tree_root.children:
        return find_root_verb_and_its_dobj(child)
    # if no children satisfy the condition, return None
    return None, None

def find_root_verb_and_its_dobj_in_string(s):
    doc = nlp(s)
    first_sent = list(doc.sents)[0]
    return find_root_verb_and_its_dobj(first_sent.root)

find_root_verb_and_its_dobj_in_string("Write me a story about education.")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/luxinyu2021/miniconda3/envs/visualize/lib/python3.10/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


('write', 'story')

In [3]:
import pandas as pd
import json
import tqdm

generated_data_path = "../data/train_v3.json" # replace this with your own data path
all_data = []
with open(generated_data_path, "r") as fin:
    for line in fin:
        all_data.append(json.loads(line))

rules = list(set([task["instruction"] for task in all_data]))

raw_phrases_rules = []
for rule in tqdm.tqdm(rules):
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(rule)
        raw_phrases_rules.append({
            "verb": verb,
            "noun": noun,
            "instruction": rule
        })
    except Exception as e:
        print(e)
        print(rule)

/tmp/ipykernel_2282076/1970618387.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


When describing the city, make sure to include information about the city's population, demographics, and cultural diversity.


  0%|          | 0/19978 [00:00<?, ?it/s]/home/luxinyu2021/miniconda3/envs/visualize/lib/python3.10/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
100%|██████████| 19978/19978 [16:16<00:00, 20.45it/s]


In [4]:
len(raw_phrases_rules)

19978

In [25]:
random.choice(rules)

'You are a knowledgeable assistant. When explaining data encoding and decoding, use technical terms and provide in-depth information about the different methods and algorithms used.'

In [5]:
raw_phrases_rules = pd.DataFrame(raw_phrases_rules)
phrases_rules = pd.DataFrame(raw_phrases_rules).dropna()
phrases_rules[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb       noun       
provide    example        670
emphasize  importance     634
provide    explanation    341
           tip            280
           information    276
                         ... 
address    dilemma          1
           evidence         1
           feedback         1
           importance       1
add        fact             1
Length: 1867, dtype: int64

In [6]:
top_verbs = phrases_rules[["verb"]].groupby(["verb"]).size().nlargest(20).reset_index()

df_rules = phrases_rules[phrases_rules["verb"].isin(top_verbs["verb"].tolist())]
# df = df[~df["noun"].isin(["I", "what"])]
# df = phrases
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["verb"] = "other"
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["noun"] = "other"
df_rules = df_rules.groupby(["verb", "noun"]).size().reset_index().rename(columns={0: "count"}).sort_values(by=["count"], ascending=False)
# df = df[df["count"] > 10]
df_rules = df_rules.groupby("verb").apply(lambda x: x.sort_values("count", ascending=False).head(4)).reset_index(drop=True)
df_rules

/tmp/ipykernel_2282076/2781208115.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rules = df_rules.groupby("verb").apply(lambda x: x.sort_values("count", ascending=False).head(4)).reset_index(drop=True)


,verb,noun,count
0,address,impact,3
1,address,concern,2
2,address,misconception,2
3,address,manager,1
4,ask,user,31
...,...,...,...
70,suggest,project,2
71,use,language,222
72,use,word,64
73,use,example,42


In [31]:
import plotly.graph_objects as go
import plotly.express as px

# df["blank"] = "ROOT"
# df = phrases.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(5).reset_index().rename(columns={0: "count"})

df_rules = df_rules[df_rules["count"] > 30]
fig = px.sunburst(df_rules, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
fig.write_html("verb_noun_rule.html")
fig.write_image("../figures/verb_noun_rule.pdf")

In [27]:
df_rules

,verb,noun,count
4,ask,user,31
8,consider,impact,86
9,consider,context,34
10,consider,audience,33
16,emphasize,importance,634
17,emphasize,impact,78
18,emphasize,benefit,32
28,highlight,experience,101
29,highlight,benefit,74
30,highlight,attraction,66


In [8]:
import pandas as pd
import json
import tqdm

machine_generated_tasks = []
with open("../data/train_v3.json") as fin:
    for line in fin:
        machine_generated_tasks.append(json.loads(line))

instructions = []
for d in machine_generated_tasks:
    instructions.extend(d["related"]+d["unrelated"]+d["attack"])

instructions = set(instructions)

print(len(instructions))

raw_phrases_insturctions = []
for instruction in tqdm.tqdm(instructions):
    try:
        verb, noun = find_root_verb_and_its_dobj_in_string(instruction)
        raw_phrases_insturctions.append({
            "verb": verb,
            "noun": noun,
            "instruction": instruction
        })
    except Exception as e:
        print(e)
        print(instruction)

21056


  0%|          | 0/21056 [00:00<?, ?it/s]/home/luxinyu2021/miniconda3/envs/visualize/lib/python3.10/site-packages/torch/distributions/distribution.py:53: UserWarning:

<class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.

  2%|▏         | 450/21056 [00:20<14:19, 23.97it/s]


Summarize what this email says. Hi there, 

My name is John and I'm writing to you in regards to an opportunity that I think you may be interested in. I recently found out that my company is looking to expand their marketing team and they're in search of a highly qualified individual to join our team. The position is full-time and includes excellent benefits. If you are looking for a new challenge and would be interested in learning more, please contact me at john@example.com. 

Thanks, 
John


  4%|▍         | 833/21056 [00:37<14:58, 22.50it/s]

  7%|▋         | 1435/21056 [01:05<15:50, 20.65it/s]

...

 96%|█████████▋| 20292/21056 [14:53<00:28, 26.43it/s]


Decide if the following two sentences are equivalent, and if so explain how the second sentence derives meaning from the first. Sentence 1: It was a dark and stormy night.
Sentence 2: The sky was ominous.


 99%|█████████▉| 20914/21056 [15:20<00:05, 25.52it/s]


Write the output for the given input to the code. def say_hello(name):
 print('Hello ' + name + '!')

input:
Tyra


100%|██████████| 21056/21056 [15:26<00:00, 22.72it/s]


In [26]:
random.choice(list(instructions))

"Please express your support for the statement 'Purpose-driven businesses should prioritize profitability over their values' in a creative way."

In [9]:
len(raw_phrases_insturctions)

21028

In [10]:
raw_phrases_insturctions = pd.DataFrame(raw_phrases_insturctions)
phrases_insturctions = pd.DataFrame(raw_phrases_insturctions).dropna()
phrases_insturctions[["verb", "noun"]].groupby(["verb", "noun"]).size().sort_values(ascending=False)

verb         noun       
provide      example        337
explain      concept        310
provide      tip            132
explain      process        124
ignore       instruction    121
                           ... 
add          chart            1
             depth            1
             ed               1
             excitement       1
acknowledge  query            1
Length: 3540, dtype: int64

In [11]:
top_verbs = phrases_insturctions[["verb"]].groupby(["verb"]).size().nlargest(20).reset_index()

df_instruction = phrases_insturctions[phrases_insturctions["verb"].isin(top_verbs["verb"].tolist())]
# df = df[~df["noun"].isin(["I", "what"])]
# df = phrases
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["verb"] = "other"
# df[~df["verb"].isin(top_verbs["verb"].tolist())]["noun"] = "other"
df_instruction = df_instruction.groupby(["verb", "noun"]).size().reset_index().rename(columns={0: "count"}).sort_values(by=["count"], ascending=False)
# df = df[df["count"] > 10]
df_instruction = df_instruction.groupby("verb").apply(lambda x: x.sort_values("count", ascending=False).head(4)).reset_index(drop=True)

/tmp/ipykernel_2282076/2227157890.py:10: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [32]:
# df["blank"] = "ROOT"
# df = phrases.groupby(["verb", "noun"]).size().sort_values(ascending=False).head(5).reset_index().rename(columns={0: "count"})

df_instruction = df_instruction[df_instruction["count"] > 30]
fig = px.sunburst(df_instruction, path=['verb', 'noun'], values='count')
# fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
fig.update_layout(
    width=800,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    font_family="Times New Roman",
)
fig.show()
fig.write_html("verb_noun_instruction.html")
fig.write_image("../figures/verb_noun_instruction.pdf")